In [ ]:
import numpy as np
from dynamice.utils.parsers import get_backbone_pdb
from idpconfgen.libs.libcalc import calc_torsion_angles
from dynamice.utils.get_sidechain import GetTorsion

In [ ]:
import os
import time

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
# set matplotlib font to be editable by Ai
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
#from Bio.PDB import PDBParser as parser
import pandas as pd

In [ ]:
cm = 1/2.54

In [ ]:
asyn_seq = 'MDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSIAAATGFVKKDQLGKNEEGAPQEGILEDMPVDPDNEAYEMPSEEGYQDYEPEA' 
drk_seq = 'MEAIAKHDFSATADDELSFRKTQILKILNMEDDSNWYRAELDGKEGLIPSNYIEMKNHD'

In [ ]:
### Production Images ###
# test pdbs histograms compare

In [ ]:
from dynamice.utils.utility import sidechain_res
def torsion_align(tor2, seq):
    # align torsions for correlation plot, returns the shared idx according tor2 (for data padded with nans)
    tor_name = {'chi1': 3, 'chi2': 4, 'chi3': 5, 'chi4': 6}
    tor2 = tor_name[tor2]
    res_num = np.array([sidechain_res[res] for res in seq])
    tor2_res = np.argwhere(res_num+2 >= tor2).flatten()
    return tor2_res
    

In [ ]:
align_idx = torsion_align('chi2', asyn_seq)

In [ ]:
val_pdb_path = ['local/training_1/reinforce/prejc_pdbs/gen_%i.pdb'%(i+1)
                for i in np.random.randint(250, size=100)]

bbs = []
for f in val_pdb_path:
    bb = next(get_backbone_pdb(f, 1))[0]
    bbtor = calc_torsion_angles(bb)
    bbs.append(bbtor)
bbs = np.array(bbs)
test_phis = np.degrees(bbs[:, 2::3])
test_psis = np.degrees(bbs[:, 0::3])
test_phis = test_phis[:, :-1].flatten()
test_psis = test_psis[:, 1:].flatten()

values = np.vstack([test_phis, test_psis])
bb_kernel = stats.gaussian_kde(values)(values)

In [ ]:
# X-EISD optimized structures

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8.7*cm, 7*cm))
sns.scatterplot(test_phis, test_psis, hue=bb_kernel, s=5, ax=ax, palette='summer',
                legend=False, linewidth = 0)
ax.set_xlabel(r'$\phi$')
ax.set_ylabel(r'$\psi$')
ax.set_xlim(-180, 180)
ax.set_ylim(-180, 180)
ax.set_xticks([-180, -90, 0, 90, 180])
ax.set_xticklabels([-180, -90, 0, 90, 180])
ax.set_yticks([-180, -90, 0, 90, 180])
ax.set_yticklabels([-180, -90, 0, 90, 180])
norm = plt.Normalize(bb_kernel.min()*1e4, bb_kernel.max()*1e4)
sm = plt.cm.ScalarMappable(cmap="summer", norm=norm)
sm.set_array([])
f.colorbar(sm, ax=ax, label='Density (1e-04)')
params = {'axes.labelsize': 8, 'xtick.labelsize': 8,
          'ytick.labelsize': 8}
plt.rcParams.update(params)
plt.tight_layout()
plt.show()

In [ ]:
plt.clf()

In [ ]:
# plot dssp
from dynamice.utils.dssp import count_ssp, ssp_per_residue
import mdtraj as md

def end_to_end(path):
    backbones = next(get_backbone_pdb(path, 1))[0]
    N_coord = backbones[1, :]
    C_coord = backbones[-2, :]
    return np.linalg.norm(N_coord - C_coord)

def compute_aspher(traj):
    l1, l2, l3 = md.principal_moments(traj)[0]
    return 1 - 3*(l1*l2+l1*l3+l2*l3)/(l1+l2+l3)**2

# secondary structure propensity
def calc_struct_features(seq_len, pdblist, props):
    ssp_pop = {'turn':[], 'bridge':[], 'helix':[], 'pi-helix':[], '310-helix':[],
               'strand':[], 'bend':[]}
    rglist = []
    reelist = []
    asplist = []
    res_ssp = {}
    count = 0
    for filename in pdblist:
        pdb = md.load(filename)
        if 'dssp' in props:
            dssp = md.compute_dssp(pdb, False)
            ss = ''.join(dssp[0])
            # for structures with helical regions
            ifhelix = count_ssp(ss, seq_len, ssp_pop, helix=False)
            count += ifhelix
            #if ifhelix: hi.append(n+250)
        if 'rg' in props:
            rglist.append(md.compute_rg(pdb)[0]*10.)
        if 'ree' in props:
            reelist.append(end_to_end(filename))
        if 'aspher' in props:
            asplist.append(compute_aspher(pdb))
    if 'dssp' in props:
        res_ssp = ssp_per_residue(ssp_pop, seq_len, count)
    return res_ssp, rglist, reelist, asplist

In [ ]:
lseq = 140

In [ ]:
dssplist_pool = {'T': [], 'E': [], 'H': [], 'Hp': [], 'H3': []}
rg_pool = []
for n in range(50):
    idxs = np.random.randint(4940, size=100)
    pdblist = ['torsions/asyn_mcsce/total/%i.pdb'%(n+1) for n in idxs]
    rg = calc_struct_features(lseq, pdblist, props=['rg'])[1]
    rg_pool.append(rg)



In [ ]:
idx = pd.read_csv('/home/oufan/Desktop/X-EISD/local/mcsce/L+E+/max_jc_pre_asyn/indices.csv', header=None)
dssplist_eisd = {'T': [], 'E': [], 'H': [], 'Hp': [], 'H3': []}
rg_eisd = []
for n in range(50):
    pdblist = ['torsions/asyn_mcsce/total/%i.pdb'%(i+1) for i in idx.iloc[np.random.randint(100)]]
    res_ssp, rg, ree, asp = calc_struct_features(lseq, pdblist, props=['rg'])    
    rg_eisd.append(rg)

    '''
    dssplist_eisd['E'].append(np.sum(res_ssp[['strand', 'bridge']], axis=1))
    dssplist_eisd['T'].append(np.sum(res_ssp[['bend', 'turn']], axis=1))
    dssplist_eisd['H'].append(res_ssp['helix'])
    dssplist_eisd['Hp'].append(res_ssp['pi-helix'])
    dssplist_eisd['H3'].append(res_ssp['310-helix'])
    '''
    
print(np.mean(np.mean(np.power(rg_eisd, 2), -1)**0.5), np.mean(np.std(rg_eisd, -1)))

In [ ]:
dssplist_rl = {'T': [], 'E': [], 'H': [], 'Hp': [], 'H3': []}
rg_rl = []
ree_rl = []
#asp_rl = []
for n in range(50):
    pdblist = ['local/training_1/reinforce/prejc_pdbs/gen_%i.pdb'%(n+1) 
               for n in np.random.randint(250, size=100)] 
    res_ssp, rg, ree, asp = calc_struct_features(lseq, pdblist, props=['dssp'])
    #rg_rl.append(rg)
    #ree_rl.append(ree)
    #asp_rl.append(asp)
    
    dssplist_rl['E'].append(np.sum(res_ssp[['strand', 'bridge']], axis=1))
    dssplist_rl['T'].append(np.sum(res_ssp[['bend', 'turn']], axis=1))
    dssplist_rl['H'].append(res_ssp['helix'])
    dssplist_rl['Hp'].append(res_ssp['pi-helix'])
    dssplist_rl['H3'].append(res_ssp['310-helix'])

In [ ]:
x = np.arange(lseq)+1
labelmap = {'E': r'$\beta$-sheet', 'T': 'Turn', 'H': r'$\alpha$-helix', 'Hp': r'$\pi$-helix', 'H3': r'$3_{10}$-helix'}
colormap = {'H': 'C2', 'T': 'C1', 'E': 'C0', 'Hp': 'C3', 'H3': 'C4'}
#cmap = {'H': 'darkseagreen', 'T': 'burlywood', 'E': 'cornflowerblue'}
cm = 1/2.54
f, ax = plt.subplots(1, 1, figsize=(8.7*cm, 7*cm))
for key in dssplist_rl:
    m = np.mean(dssplist_rl[key], axis=0)
    ax.errorbar(x, m, yerr=np.std(dssplist_rl[key], axis=0),
               label=labelmap[key], c=colormap[key])
ax.set_ylim(top=0.8)
ax.set_xlabel('Residue Number')
ax.set_ylabel('Secondary Structure Propensity')
ax.legend(loc='upper center', ncol=3, frameon=False, fontsize=7)

params = {'axes.labelsize': 8, 'xtick.labelsize': 8,
          'ytick.labelsize': 8, 'lines.linewidth': 0.5}
plt.rcParams.update(params)
plt.tight_layout()
plt.show()

In [ ]:
#plot rg distribution
f, ax = plt.subplots(1, 1, figsize=(8.7*cm, 7*cm))
ax.hist(np.reshape(rg_pool, -1), bins=np.arange(15, 85, 2), density=True, edgecolor='navy', fill=False,
         label='Original pool')
ax.hist(np.reshape(rg_eisd[:10], -1), bins=np.arange(15, 85, 2)+0.2, density=True, edgecolor='darkorange', fill=False,
         label='Reweighting')
#facecolor='none', edgecolor='black')
ax.hist(np.reshape(rg_rl[:10], -1), bins=np.arange(15, 85, 2)+0.4, density=True, edgecolor='green', alpha=0.3,
         label='RL model')

ax.set_xlabel(r'$R_g  (\AA)$')
ax.set_ylabel('Propensity')
ax.legend(frameon=False, fontsize=7)

params = {'axes.labelsize': 8, 'xtick.labelsize': 8,
          'ytick.labelsize': 8, 'lines.linewidth': 0.5}
plt.rcParams.update(params)
plt.tight_layout()
plt.show()

In [ ]:
# plot SAXS exp bc curve
data_path = '/home/oufan/Desktop/X-EISD/data/asyn/'
saxs_exp = pd.read_csv(data_path + 'experimental_data/asyn_saxs_exp.dat')
saxs_pool = pd.read_csv(data_path + 'back_calc_data/MCSCE/asyn_SAXS.txt', header=None, index_col=0)
saxs_rl = pd.read_csv(data_path + 'back_calc_data/ML/rl_LE_saxs.txt', header=None, index_col=0)

In [ ]:
df1 = pd.read_csv('/home/oufan/Desktop/X-EISD/local/mcsce/L+E+/max_jc_pre_asyn/indices.csv', header=None)
saxs_eisd = []
for n in range(50):
    d = saxs_pool.iloc[df1.iloc[np.random.randint(100)].values].mean(axis=0)
    saxs_eisd.append(d)

In [ ]:
rl = []
for n in range(50):
    d = saxs_rl.iloc[np.random.randint(250, size=100)].mean(axis=0)
    rl.append(d)

In [ ]:
#plot SAXS 
f, ax = plt.subplots(1, 1, figsize=(8.7*cm, 7.5*cm))

x = saxs_exp['index'].values[:1000]

ax.errorbar(x, saxs_exp['value'].values[:1000], yerr=saxs_exp['error'].values[:1000], 
            label='Experimental', color='C0')
ax.errorbar(x, np.mean(saxs_eisd, axis=0)[:1000], yerr=np.std(saxs_eisd, axis=0)[:1000], 
            label='Reweighting', linestyle='--', color='C1')
ax.errorbar(x, np.mean(rl, axis=0)[:1000], yerr=np.std(rl, axis=0)[:1000], label='RL model',
           linestyle='-.', color='C2')

#ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_xlabel(r'$Q({\AA}^{1})$')
ax.set_ylabel('I(Q)')
ax.legend(frameon=False, fontsize=7)

params = {'axes.labelsize': 8, 'xtick.labelsize': 8,
          'ytick.labelsize': 8, 'lines.linewidth': 0.5}
plt.rcParams.update(params)
plt.tight_layout()
plt.show()